Import Dependancies

In [1]:
# Dependencies
from config import (gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#import cleaned county data
county_data = pd.read_csv("../Matthew/Data_csv/state_county.csv")
#county_data.head()

,Unnamed: 0,State,County,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)"
0,1,Alabama,NaN,69.1,76.9,69.4,77.1,69.9,77.2,71.0,77.2,71.5,77.3,72.4,77.8,3.3,0.9
1,2,Alabama,Autauga,68.1,77.0,68.5,77.0,69.3,77.3,71.7,77.6,72.5,77.4,73.3,78.8,5.2,1.8
2,3,Alabama,Baldwin,71.1,78.8,71.9,79.2,72.4,79.3,73.9,79.2,73.8,79.7,75.0,80.3,3.8,1.5
3,4,Alabama,Barbour,66.8,76.0,66.7,76.7,68.4,76.9,70.0,76.2,70.5,77.0,72.2,77.2,5.3,1.1
4,5,Alabama,Bibb,67.3,76.6,68.4,76.2,69.0,75.9,70.3,76.5,70.5,76.8,71.5,76.8,4.2,0.2


In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Generate Random Cities in the USA

In [10]:
# Range of latitudes and longitudes
lat_range = (25, 50)
lng_range = (-125,-65)

# List for holding lat_lngs and cities
lat_lngs = []
random_cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in random_cities:
        random_cities.append(city)

# Print the city count to confirm sufficient count
#random_cities

# Gather Geo and County Data

In [20]:
lat = []
lon = []
county = []

# Build the endpoint URL
target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"
test_url = f"https://maps.googleapis.com/maps/api/geocode/json?address=SanAntonio&key={gkey}"

test_data = requests.get(test_url).json()
test_data['results'][0]['address_components'][1]['long_name'].split()[0]

'Bexar'

In [21]:
# Run a request to endpoint and convert result to json
#geo_data = requests.get(target_url).json()

cities = []
county = []
lat = []
lon = []

#verify_us = geo_data['results'][0]['address_components'][3]['short_name']

# Print the json
#grab county from the json
for city in random_cities[:100]:
    # create query url and get json
    query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    geo_data = requests.get(query_url).json()
        
    #if the city has data then populate lists
    if (geo_data['status'] == 'OK'):
    
        cities.append(city),
        county.append(geo_data['results'][0]['address_components'][1]['long_name'].split()[0])
        lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        

IndexError: list index out of range

In [ ]:
lat

In [ ]:
lon

In [ ]:
county

In [9]:
returned_cities

NameError: name 'returned_cities' is not defined